In [ ]:
from initialize import *
initialize()

# Test Energy Imbalance Handling

In [ ]:
#Check if admin (user that deployed the contract) is an admin by default:
PowerChain.execute(METHODS.ADD_STORAGE_UNIT,toAddress(unit1.address),toAddress(user1.address),account=admin)
printResult("Admin makes unit1 a storage unit with owner user1",
            any(u[1] == unit1.address for u in PowerChain.call(METHODS.GET_STORAGE_UNIT_INFO)), True)

printResult("User1 produces 10 kWh in unit1 and unit1 reports the production",
            PowerChain.execute(METHODS.ENERGY_PRODUCED,toAddress(user1.address),10000,account=unit1), "")

printResult("Unit1 has now 10 kWh",
            PowerChain.call(METHODS.GET_STORAGE_UNIT_ENERGY,toAddress(unit1.address)),10000)

printResult("User1 has now 10 ENT",
            PowerChain.call(METHODS.BALANCE_ENT,account=user1),[toInternalNumber(10),0])

printResult("Unit1 lost a battery, has now 9 kWh and reports the actual energy",
            PowerChain.execute(METHODS.REPORT_ACTUAL_ENERGY,9000,account=unit1),"")

rates = PowerChain.call(METHODS.GET_ENERGY_RATES)
printResult("The cost to mint tokens now is 0.05 ENT / kWh",
            rates[0],toInternalNumber(0.05))
printResult("The cost to burn tokens now is 0.05 ENT / kWh",
            rates[1],toInternalNumber(0.05))

printResult("User1 produces 10 kWh in unit1 and unit1 reports the production",
            PowerChain.execute(METHODS.ENERGY_PRODUCED,toAddress(user1.address),10000,account=unit1), "")
printResult("User1 has now 19.5 ENT, 9.5 ENT rewarded due to mint cost",
            PowerChain.call(METHODS.BALANCE_ENT,account=user1),[toInternalNumber(19.5),0])
printResult("Unit1 has now 19 kWh",
            PowerChain.call(METHODS.GET_STORAGE_UNIT_ENERGY,toAddress(unit1.address)),19000)

printResult("User1 starts a consumption session worth 19.5 ENT with unit1",
            "session started" in PowerChain.execute(METHODS.START_CONSUMPTION_SESSION,toAddress(unit1.address),toInternalNumber(19.5),account=user1),True)
printResult("There is one consumption session for user1",
            len(PowerChain.call(METHODS.GET_CONSUMPTION_SESSIONS,account=user1)), 1)
printResult("Consumtion session of unit1 with user1 should now have 19.5 / 1.05 = 18.571 kWh",
            PowerChain.call(METHODS.GET_CONSUMPTION_SESSION_ENERGY,toAddress(unit1.address),account=user1), 18571)
printResult("User1 has now 19.5 ENT locked",
            PowerChain.call(METHODS.BALANCE_ENT,account=user1),[0,toInternalNumber(19.5)])

printResult("User1 consumes 18.571kWh from the consumption session with unit1 and unit1 reports it",
            PowerChain.execute(METHODS.ENERGY_CONSUMED,toAddress(user1.address),18571,account=unit1) , "")
printResult("No active consumption sessions for user1",
            len(PowerChain.call(METHODS.GET_CONSUMPTION_SESSIONS,account=user1)), 0)

rates = PowerChain.call(METHODS.GET_ENERGY_RATES)
printResult("The cost to mint tokens now is 0 ENT / kWh",
            rates[0],0)
printResult("The cost to burn tokens now is 0 ENT / kWh",
            rates[1],0)

print("After the consumtion of the 10 kWh system was set to equilibrium again and the remaining session had no extra costs")
printResult("User1 should have 0.429 ENT token returned from the consumption session",PowerChain.call(METHODS.BALANCE_ENT,account=user1),[toInternalNumber(0.429),0])
printResult("User1 starts a consumption session worth 0.429 ENT with unit1",
            "session started" in PowerChain.execute(METHODS.START_CONSUMPTION_SESSION,toAddress(unit1.address),toInternalNumber(0.429),account=user1),True)
printResult("User1 consumes 0.429kWh from the consumption session with unit1 and unit1 reports it",
            PowerChain.execute(METHODS.ENERGY_CONSUMED,toAddress(user1.address),429,account=unit1) , "")
printResult("No active consumption sessions for user1",
            len(PowerChain.call(METHODS.GET_CONSUMPTION_SESSIONS,account=user1)), 0)
printResult("Total network energy is 0 kWh",
            PowerChain.call(METHODS.GET_TOTAL_KWH),0)